# Scenario A - Noise Level Variation (individual dataset)

This file is used to zoom in and experiment on on a previously generated dataset.

The model used in the inference of the parameters is formulated as follows: 

\begin{equation}
\large y = f(x) = \sum\limits_{m=1}^M \big[A_m \cdot e^{-\frac{(x-\mu_m)^2}{2\cdot\sigma_m^2}}\big] + \epsilon
\end{equation}

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymc3 as pm
import arviz as az

#az.style.use('arviz-darkgrid')

print('Running on PyMC3 v{}'.format(pm.__version__))

## Import local modules

In [ ]:
import datetime
import os
import sys
sys.path.append('../../modules')
import datagen as dg
import models as mdl
import results as res
import figures as fig
import settings as cnf

## Local configuration

In [ ]:
# dataset to load
ndata = '06'
input_datafile = './input_datasets/dataset_'+ str(ndata) + '.csv' 
input_peakfile = './input_datasets/peakinfo.csv' 

# output for results and images
out_path = './output_' + str(ndata)
file_basename = out_path + '/scenario_noise_' + str(ndata)

# if dir does not exist, create it
if not os.path.exists(out_path):
    os.makedirs(out_path)

conf = {}

# scenario name
conf['scenario'] = 'noise variation'

# initialization method for sampler ('jitter+adapt_diag'/'advi+adapt_diag'/'adapt_diag')
conf['init_mode'] = 'adapt_diag'
    
# probabilistic model (priors)
conf['prior_model'] = 'lognormal'

# provide peak positions to the model as testvalues ('yes'/'no')
conf['peak_info'] = 'yes'

# absolute peak shift (e.g. 2%(4), 5%(10) or 10%(20) of X-min.)
conf['peak_shift'] = 10

# model mode ('train')
conf['model_mode'] = 'train'

# number of times to run inference engine on dataset
conf['niter'] = 2

# number of cores to run
conf['ncores'] = 2

# number of samples per chain
conf['nsamples'] = 2000

## Save configuration

In [ ]:
cnf.save(out_path, conf)

# Load data and plot

In [ ]:
ldata  = []
lpeaks = []

# load data from disk
print("loading file {0}".format(input_datafile))
df = pd.read_csv(input_datafile)
ldata.append(df)
print("loading peakinfo {0}".format(input_peakfile))
pks = np.loadtxt(input_peakfile, delimiter=',')
pks += conf['peak_shift']
lpeaks.append(pks[int(ndata)-1])

In [ ]:
# plot datasets
fig.plot_datasets(ldata, lpeaks, dims=(1,1), figure_size=(8,6), savefig='yes', fname=file_basename)

# Initialize models and run inference

In [ ]:
niter = conf['niter']

# expand dataframe and peak list
for i in range(niter-1):
    ldata.append(ldata[0])
    lpeaks.append(lpeaks[0])
    
# convert pandas data to numpy arrays
x_val = np.array(ldata[0].columns.to_list(), dtype='float32')

# store dataset y-values in list
cols = ldata[0].columns
y_val = [ldata[i][cols].values for i in range(len(ldata))]

In [ ]:
# initialize models and run inference
models = []
traces = []

nsamples    = conf['nsamples']
init_mode   = conf['init_mode']
ncores      = conf['ncores']
prior_model = conf['prior_model']

for i in range(len(ldata)):
        if conf['peak_info'] == 'yes':
            plist = lpeaks[i].flatten()
            plist.sort()
            model_g = mdl.model_gauss(xvalues=x_val, observations=y_val[i], npeaks=3, 
                                      mu_peaks=plist, pmodel=prior_model)
        else:
            model_g = mdl.model_gauss(xvalues=x_val, observations=y_val[i], npeaks=3,
                                      pmodel=prior_model) 
        models.append(model_g)

        with model_g:
            if conf['model_mode'] == 'train':
                print("running inference on dataset #{0}/{1}".format(i+1,len(ldata)))
                trace_g = pm.sample(nsamples, init=init_mode, cores=ncores)
                traces.append(trace_g)
                # save inference results
                pm.backends.text.dump(out_path + '/traces_%02d' % (i+1), trace_g)

# Model visualization

In [ ]:
pm.model_to_graphviz(models[0])

In [ ]:
# save model figure as image
img = pm.model_to_graphviz(models[0])
img.render(filename=file_basename + '_model', format='png');

# Collect results and save

In [ ]:
# posterior predictive traces
ppc = [pm.sample_posterior_predictive(traces[i], samples=500, model=models[i]) for i in range(len(traces))]

In [ ]:
# various plots to inspect the inference results
varnames = ['amp', 'mu', 'sigma', 'epsilon']

#az.summary(traces[0], varnames)
#az.plot_trace(traces[2], varnames, compact=True);
#az.plot_trace(traces[2], varnames, divergences='top');
#az.plot_autocorr(traces[0], varnames);
#az.plot_posterior(traces[2], varnames);

#for idx, trace in enumerate(traces):
#    az.plot_forest(trace, var_names = varnames, r_hat=True, ess=True);

In [ ]:
#az.plot_trace(traces[0], varnames, compact=True);

In [ ]:
if conf['model_mode'] == 'train':
    # collect the results and display
    df = res.get_results_summary(varnames, traces, ppc, y_val)
else:
    # load results from disk
    df = pd.read_csv(file_basename + '.csv')
df

In [ ]:
if conf['model_mode'] == 'train':
    # save results to .csv
    df.to_csv(file_basename + '.csv', index=False)

# Plot posterior

In [ ]:
fig.plot_posterior(x_val, ldata, traces, ppc, dims=(int(niter/2),2), figure_size=(16,12), savefig='yes', 
                       fname=file_basename, showpeaks='no')

In [ ]:
fig.plot_posterior(x_val, ldata, traces, ppc, dims=(int(niter/2),2), figure_size=(16,12), savefig='yes', 
                       fname=file_basename, showpeaks='yes')

In [ ]:
cnf.close(out_path)